In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from google.colab import files
uploaded = files.upload()

Saving CPI_data.csv to CPI_data.csv


In [28]:
# Load the data
df = pd.read_csv('CPI_data.csv', index_col='date', parse_dates=True)

In [14]:
# df["date"] = pd.to_datetime(df["date"], format="%d/%m/%Y")

In [ ]:
plt.plot(df["date"],df["CPI"])
plt.xlabel('DATE')
plt.ylabel('CPI')
plt.title("Sample graph")
plt.show()

In [29]:
train_set = df.iloc[:456]
test_set = df.iloc[456:]

In [23]:
from statsmodels.tsa.seasonal import seasonal_decompose

results = seasonal_decompose(df['CPI'])
results.plot()

In [30]:
# Normalize the data
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(train_set)

scaled_train = scaler.transform(train_set)
scaled_test = scaler.transform(test_set)

In [34]:
from keras.preprocessing.sequence import TimeseriesGenerator

# Define generator
n_input = 12
n_features = 1
generator = TimeseriesGenerator(scaled_train, scaled_train, length = n_input, batch_size = 1)

In [35]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

# Create the LSTM model
model = Sequential()
model.add(LSTM(100, activation = 'relu', input_shape=(n_input, n_features)))
model.add(Dense(1))

In [36]:
# Compile the model
model.compile(optimizer='adam', loss='mse')

In [37]:
# fit the model
model.fit(generator, epochs = 50)

Epoch 1/50
444/444 [==============================] - 6s 10ms/step - loss: 0.0136
Epoch 2/50
444/444 [==============================] - 3s 8ms/step - loss: 2.8970e-04
Epoch 3/50
444/444 [==============================] - 3s 7ms/step - loss: 1.7337e-04
Epoch 4/50
444/444 [==============================] - 4s 10ms/step - loss: 2.4650e-04
Epoch 5/50
444/444 [==============================] - 3s 7ms/step - loss: 2.0353e-04
Epoch 6/50
444/444 [==============================] - 3s 7ms/step - loss: 2.7953e-04
Epoch 7/50
444/444 [==============================] - 4s 10ms/step - loss: 2.1579e-04
Epoch 8/50
444/444 [==============================] - 3s 8ms/step - loss: 3.2390e-04
Epoch 9/50
444/444 [==============================] - 3s 8ms/step - loss: 2.5118e-04
Epoch 10/50
444/444 [==============================] - 6s 13ms/step - loss: 2.1090e-04
Epoch 11/50
444/444 [==============================] - 3s 8ms/step - loss: 2.0577e-04
Epoch 12/50
444/444 [==============================] - 3s 8ms/s

In [38]:
last_train_batch = scaled_train[-12:]

In [39]:
last_train_batch = last_train_batch.reshape((1, n_input, n_features))

In [40]:
model.predict(last_train_batch)

1/1 [==============================] - 0s 227ms/step


array([[1.0113301]], dtype=float32)

In [41]:
scaled_test[0]

array([1.00620049])

In [43]:
test_predictions = []

first_eval_batch = scaled_train[-n_input:]
current_batch = first_eval_batch.reshape((1, n_input, n_features))

for i in range(len(test_set)):
  # get the prediction value for the first batch
  current_pred = model.predict(current_batch)[0]

  # append the prediction into the array
  test_predictions.append(current_pred)

  # use the predcition to update the batch and remove the first value
  current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)



1/1 [==============================] - 0s 30ms/step


In [44]:
test_predictions

[array([1.0113301], dtype=float32),
 array([1.0199959], dtype=float32),
 array([1.0284125], dtype=float32),
 array([1.0364176], dtype=float32),
 array([1.0438066], dtype=float32),
 array([1.0509164], dtype=float32),
 array([1.057865], dtype=float32),
 array([1.0647762], dtype=float32),
 array([1.0716985], dtype=float32),
 array([1.0785931], dtype=float32),
 array([1.085526], dtype=float32),
 array([1.0925018], dtype=float32),
 array([1.099596], dtype=float32),
 array([1.1067123], dtype=float32),
 array([1.1138483], dtype=float32),
 array([1.1209925], dtype=float32),
 array([1.1281369], dtype=float32),
 array([1.135282], dtype=float32),
 array([1.1424296], dtype=float32),
 array([1.1495813], dtype=float32),
 array([1.1567392], dtype=float32),
 array([1.1639043], dtype=float32),
 array([1.1710765], dtype=float32),
 array([1.1782568], dtype=float32),
 array([1.1854447], dtype=float32),
 array([1.1926394], dtype=float32),
 array([1.1998401], dtype=float32),
 array([1.2070464], dtype=float3

In [46]:
true_predictions = scaler.inverse_transform(test_predictions)

In [47]:
true_predcitions

array([[248.73400686],
       [250.19869172],
       [251.62124579],
       [252.97426692],
       [254.22313976],
       [255.42484465],
       [256.59928843],
       [257.76740553],
       [258.93741662],
       [260.10273306],
       [261.27451722],
       [262.45355489],
       [263.65262031],
       [264.85541322],
       [266.06153065],
       [267.26903834],
       [268.47656618],
       [269.68423506],
       [270.89230691],
       [272.10108396],
       [273.31090875],
       [274.52194245],
       [275.73418507],
       [276.94777765],
       [278.16267989],
       [279.37871045],
       [280.59576859],
       [281.81377372],
       [283.03258479],
       [284.25208092],
       [285.47222181],
       [286.69296715],
       [287.91423636],
       [289.13598913],
       [290.35810458],
       [291.58056256]])

In [48]:
import pickle

# save the trained model to a file
with open('trained_lstm_model.pkl', 'wb') as f:
    pickle.dump(model, f)

files.download('trained_lstm_model.pkl')
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
test_set['Predictions'] = true_predictions
test_set.plot(figsize=(12,6))

In [ ]:
from sklearn.metrics import mean_square_error
from math import sqrt

rmse = sqrt(mean_square_error(test_set['CPI'], test_set['Predictions']))
print(rmse)